# Breaching privacy

This notebook does the same job as the cmd-line tool `simulate_breach.py`, but also directly visualizes the user data and reconstruction

In [1]:
import torch
import hydra
from omegaconf import OmegaConf
%load_ext autoreload
%autoreload 2

import breaching
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

### Initialize cfg object and system setup:

This will print out all configuration options. 
There are a lot of possible configurations, but there is usually no need to worry about most of these. Below, a few options are printed.

Choose `case/data=` `shakespeare`, `wikitext`over `stackoverflow` here:

In [2]:
with hydra.initialize(config_path="config"):
    cfg = hydra.compose(config_name='cfg', overrides=["case/data=wikitext", "case/server=malicious-transformer",
                                                      "case.model=transformer3p",
                                                      "attack=decepticon"])
    print(f'Investigating use case {cfg.case.name} with server type {cfg.case.server.name}.')
          
device = torch.device(f'cuda:0') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=torch.float)
setup

Investigating use case single_imagenet with server type malicious_transformer_parameters.


{'device': device(type='cpu'), 'dtype': torch.float32}

### Modify config options here

You can use `.attribute` access to modify any of these configurations:

In [52]:
cfg.case.user.num_data_points = 16 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [64] # This is the sequence length

cfg.case.data.tokenizer = "gpt2"

cfg.case.server.has_external_data = True

cfg.case.server.param_modification.v_length = 6*3

cfg.case.server.param_modification.eps = 1e-4
cfg.case.server.param_modification.imprint_sentence_position = 0
cfg.case.server.param_modification.softmax_skew = 10000000
cfg.case.server.param_modification.sequence_token_weight = 1

### Instantiate all parties

In [53]:
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup)
attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Reusing dataset wikitext (/home/jonas/data/wikitext/wikitext-103-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)
Model architecture transformer3p loaded with 11,095,345 parameters and 0 buffers.
Overall this is a data ratio of   10835:1 for target shape [16, 64] given that num_queries=1.
User (of type UserSingleStep) with settings:
    Number of data points: 16

    Threat model:
    User provides labels: False
    User provides buffers: False
    User provides number of data points: True

    Data:
    Dataset: wikitext
    user: 1
    
        
Server (of type MaliciousTransformerServer) with settings:
    Threat model: Malicious (Parameters)
    Number of planned queries: 1
    Has external/public data: True

    Model:
        model specification: transformer3p
        model state: default
        

    Secrets: {}
  

### Simulate an attacked FL protocol

True user data is returned only for analysis

In [54]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

torch.Size([96, 1536])
torch.Size([96, 1536])
torch.Size([96, 1536])
Computing feature distribution before the linear1 layer from external data.
Feature mean is 0.007005429361015558, feature std is 1.0136948823928833.


In [55]:
user.print(true_user_data)

 The Tower Building of the Little Rock Arsenal, also known as U.S. Arsenal Building, is a building located in MacArthur Park in downtown Little Rock, Arkansas. Built in 1840, it was part of Little Rock's first military installation. Since its decommissioning, The Tower Building has housed two museums. It
 was home to the Arkansas Museum of Natural History and Antiquities from 1942 to 1997 and the MacArthur Museum of Arkansas Military History since 2001. It has also been the headquarters of the Little Rock Æsthetic Club since 1894. 
 The building receives its name from its distinct octagonal tower. Besides being the last
 remaining structure of the original Little Rock Arsenal and one of the oldest buildings in central Arkansas, it was also the birthplace of General Douglas MacArthur, who became the supreme commander of US forces in the South Pacific during World War II. It was also the starting place of the Camden Expedition. In 2011 it was named as one of
 the top 10 attractions in th

# Reconstruct user data

In [56]:
attacker.cfg.sentence_algorithm ="dynamic-threshold"

In [57]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

Recovered tokens [[11, 12, 13, 29, 31, 50, 82, 198, 220, 228, 257, 262, 264, 278, 284, 285, 286, 287, 290, 291, 301, 307, 309, 317, 318, 319, 326, 329, 337, 339, 340, 347, 351, 352, 355, 357, 366, 371, 373, 376, 379, 383, 393, 407, 416, 422, 423, 428, 441, 465, 468, 471, 474, 475, 477, 484, 503, 504, 508, 511, 513, 517, 530, 543], [547, 550, 554, 555, 561, 573, 587, 597, 611, 617, 632, 635, 636, 642, 657, 661, 663, 685, 691, 705, 714, 717, 720, 734, 739, 764, 788, 812, 815, 832, 835, 837, 838, 852, 867, 869, 871, 884, 887, 938, 959, 973, 1011, 1022, 1028, 1037, 1109, 1114, 1141, 1181, 1201, 1204, 1230, 1248, 1267, 1279, 1294, 1295, 1315, 1353, 1363, 1438, 1444, 1450], [1455, 1478, 1502, 1541, 1542, 1550, 1566, 1568, 1578, 1605, 1625, 1630, 1636, 1664, 1705, 1708, 1742, 1745, 1748, 1749, 1750, 1757, 1810, 1811, 1812, 1829, 1900, 1908, 1936, 1956, 1957, 1965, 2071, 2074, 2098, 2113, 2158, 2159, 2162, 2236, 2237, 2286, 2361, 2366, 2378, 2391, 2406, 2411, 2422, 2427, 2488, 2520, 2540, 2581

METRICS: | Accuracy: 0.6836 | S-BLEU: 0.47 | FMSE: 1.5450e-01 | 
 G-BLEU: 0.45 | ROUGE1: 0.76| ROUGE2: 0.46 | ROUGE-L: 0.68| Token Acc: 99.22% | Label Acc: 99.22%
 The Tower Building for the Little. Arsenal, also> as U 5S. located Building, is a the in in expel Park in downtown Little Rock, part. Built the @, it was Rock of Little Rock's first militarysey with Since known decommissioning, The Tower its has housed two museums@ the
 was of to @ Arkansas It. Natural History and Antiquities from the to 1997 1942� MacArthur Museum of Arkansas Military But since November., has also been in headquarters on the Little Rock � companyst oct Club United 1894 home 
 The building receives- name from its distinct Historyagonal tower. Rock installation the its
 was of by the federal Little, Arsenal as one forces the remaining buildings owned and In, a handful also the.. General structure MacArthur, who became the supreme of Civil US horizon in the Rock Pacific during World War II. It was consisting n

In [58]:
reconstructed_user_data, stats = attacker.reconstruct([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)

metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

Recovered tokens [[11, 12, 13, 29, 31, 50, 82, 198, 220, 228, 257, 262, 264, 278, 284, 285, 286, 287, 290, 291, 301, 307, 309, 317, 318, 319, 326, 329, 337, 339, 340, 347, 351, 352, 355, 357, 366, 371, 373, 376, 379, 383, 393, 407, 416, 422, 423, 428, 441, 465, 468, 471, 474, 475, 477, 484, 503, 504, 508, 511, 513, 517, 530, 543], [547, 550, 554, 555, 561, 573, 587, 597, 611, 617, 632, 635, 636, 642, 657, 661, 663, 685, 691, 705, 714, 717, 720, 734, 739, 764, 788, 812, 815, 832, 835, 837, 838, 852, 867, 869, 871, 884, 887, 938, 959, 973, 1011, 1022, 1028, 1037, 1109, 1114, 1141, 1181, 1201, 1204, 1230, 1248, 1267, 1279, 1294, 1295, 1315, 1353, 1363, 1438, 1444, 1450], [1455, 1478, 1502, 1541, 1542, 1550, 1566, 1568, 1578, 1605, 1625, 1630, 1636, 1664, 1705, 1708, 1742, 1745, 1748, 1749, 1750, 1757, 1810, 1811, 1812, 1829, 1900, 1908, 1936, 1956, 1957, 1965, 2071, 2074, 2098, 2113, 2158, 2159, 2162, 2236, 2237, 2286, 2361, 2366, 2378, 2391, 2406, 2411, 2422, 2427, 2488, 2520, 2540, 2581

ValueError: not enough values to unpack (expected 2, got 1)

In [31]:
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

user.print(reconstructed_user_data)

METRICS: | Accuracy: 0.8086 | S-BLEU: 0.75 | FMSE: 1.0791e-01 | 
 G-BLEU: 0.71 | ROUGE1: 0.84| ROUGE2: 0.76 | ROUGE-L: 0.82| Token Acc: 97.27% | Label Acc: 97.27%
 supreme Tower Building of the Little Rock Arsenal, also known as U.S installation
 Arsenal Building place is a building located in MacArthur Park in downtown Little Rock, through
 Thirty starting constructed 1840, it was part of Little Rock's first military Built
. Since its decommissioning, The Tower Building has housed two museums. last
 was home to the Rock Museum of Natural History and Antiqu
 from 1942 to,
 and the MacArthur Museum of Arkansas Military History since 2001. It has also been of
 headquarters of the Little Rock Æsthetic Club since 1894.  central
 The building receives its name from its distinct octagonal tower. Besides being the in
 remaining structure of the original Little Rock in It one of the oldest buildings, also
 Arkansas, it was also the birthplace of General Douglas MacArthurities who became the.
 

 Natural
 the state on their way to the newly Rock supreme Territory. who @- two
 the Thirty, oldest original Little Military� one 1840 of as Besides also to,
 the structure 1997 last remaining Little of Camden In Building central.. Rock �st
 The building receives its name from its distinct octagonal tower. its being the 1942
. top place forces Rock building was Pacific History World since downtown from response buildingsmission
 and the through Museum of Arkansas the History since 2001. It has also been Rock
 The Tower Building of the Little Rock Arsenal, also known as the.S first
 Arsenal Tower MacArthur is the of the in MacArthur Park in attractions Little Rock, Little
 perceived dangers of frontier life and fears of one many II Americans who werehetic Little
 arsenal was of at the request of Governor James Sevier Conway in it to military
 Arkansas, it was also General birthplace of established Douglas Since the in became the@
 the home the of,,94
 and Club,'named has passing also
. starting constructed Museum aing housed part Arkansas Oklahoma inities U.  Expedition
 was Building it of of Arkansas located installation MacArthurs Antiqu 18 in museums to of

In [52]:
reconstructed_user_data, stats = attacker.reconstruct_single_sentence([server_payload], [shared_data], 
                                                      server.secrets, dryrun=cfg.dryrun)
user.print(reconstructed_user_data)
metrics = breaching.analysis.report(reconstructed_user_data, true_user_data, [server_payload], 
                                    server.model, cfg_case=cfg.case, setup=setup)

Recovered tokens [[0, 1, 2, 5, 6, 7, 8, 10, 11, 12, 14, 16, 17, 18, 19, 22], [25, 26, 29, 30, 31, 32, 34, 35, 38, 40, 43, 50, 51, 56, 61, 62], [63, 64, 71, 72, 75, 91, 98, 104, 108, 153, 154, 184, 245, 277, 279, 291], [294, 310, 319, 351, 400, 405, 456, 494, 566, 572, 605, 652, 794, 846, 872, 900], [926, 940, 963, 993, 1084, 1263, 1404, 1407, 1495, 1672, 1936, 2108, 2173, 2195, 2211, 2421], [2777, 2971, 3158, 3406, 3688, 3690, 3807, 3851, 4046, 5231, 5371, 5470, 5656, 5670, 6084, 6489], [6567, 8107, 8323, 8716, 8766, 9564, 11116, 13287, 15821, 18252, 18637, 21489, 21964, 22724, 23714, 24378], [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 11, 11, 262, 329, 649, 15809], [5, 5, 5, 5, 5, 5, 6, 6, 11, 11, 11, 11, 11, 11, 11, 22], [0, 5, 5, 5, 6, 6, 7, 7, 11, 11, 12, 22, 22, 22, 35, 50], [0, 5, 6, 7, 7, 11, 12, 17, 22, 35, 35, 50, 50, 566, 1084, 1936], [0, 5, 6, 7, 11, 12, 17, 19, 22, 35, 494, 566, 846, 1084, 1936, 6489], [0, 8, 12, 17, 19, 31, 50, 291, 494, 566, 846, 1084, 1936, 6084, 6489, 18637], [0, 5,